# 🖥️💫 Interface Moderna - NeuroTranslator PT-EN

Este notebook desenvolve a **interface gráfica moderna** para o NeuroTranslator usando:
- **React + Electron** para aplicação desktop
- **CustomTkinter** para interface Python nativa
- **Design responsivo** e moderno
- **Integração completa** com todos os componentes
- **Controles em tempo real** para áudio, vídeo e tradução

## Arquitetura da Interface:
```
┌─────────────────────────────────────────┐
│           NEUROTRANSLATOR               │
├─────────────────┬───────────────────────┤
│   📹 Vídeo      │  🎤 Controles Áudio   │
│   (Webcam)      │  ▶️ ⏸️ ⏹️ 🔊         │
├─────────────────┼───────────────────────┤
│  📝 Texto Original                      │
├─────────────────────────────────────────┤
│  🌐 Tradução                            │
├─────────────────────────────────────────┤
│  ⚙️ Configurações │ 📊 Estatísticas     │
└─────────────────────────────────────────┘
```

In [ ]:
# Importações para interface moderna
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import customtkinter as ctk
from PIL import Image, ImageTk, ImageDraw
import cv2
import numpy as np
import threading
import time
from datetime import datetime
import json
import os
import queue

# Importar sistema NeuroTranslator
import sys
sys.path.append('../src')

# Configurações visuais
ctk.set_appearance_mode("dark")  # "dark" ou "light"
ctk.set_default_color_theme("blue")  # "blue", "green", "dark-blue"

print("🎨 Bibliotecas de interface carregadas!")
print("💫 Tema escuro ativado!")

## 1. Classe Principal da Interface

In [ ]:
class NeuroTranslatorGUI:
    """
    Interface gráfica principal do NeuroTranslator
    """
    def __init__(self):
        # Configurações da janela
        self.window_width = 1200
        self.window_height = 800
        
        # Estado da aplicação
        self.is_running = False
        self.current_frame = None
        self.translation_history = []
        
        # Filas para comunicação entre threads
        self.gui_queue = queue.Queue()
        
        # Criar janela principal
        self.root = ctk.CTk()
        self.setup_main_window()
        
        # Criar componentes da interface
        self.create_widgets()
        
        # Inicializar sistema NeuroTranslator (simulado)
        self.neurotranslator = None
        
        print("🖥️ Interface NeuroTranslator inicializada!")
    
    def setup_main_window(self):
        """
        Configura janela principal
        """
        self.root.title("🧠 NeuroTranslator PT-EN | Real-Time Translation")
        self.root.geometry(f"{self.window_width}x{self.window_height}")
        
        # Centralizar janela
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        x = (screen_width - self.window_width) // 2
        y = (screen_height - self.window_height) // 2
        self.root.geometry(f"{self.window_width}x{self.window_height}+{x}+{y}")
        
        # Configurar protocolo de fechamento
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        
        # Ícone da aplicação (simulado)
        try:
            # self.root.iconbitmap("../assets/icon.ico")
            pass
        except:
            pass
    
    def create_widgets(self):
        """
        Cria todos os widgets da interface
        """
        # Frame principal
        self.main_frame = ctk.CTkFrame(self.root)
        self.main_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Criar seções da interface
        self.create_header()
        self.create_video_section()
        self.create_translation_section()
        self.create_controls_section()
        self.create_status_section()
    
    def create_header(self):
        """
        Cria cabeçalho da aplicação
        """
        header_frame = ctk.CTkFrame(self.main_frame)
        header_frame.pack(fill="x", pady=(0, 10))
        
        # Título principal
        title_label = ctk.CTkLabel(
            header_frame,
            text="🧠 NeuroTranslator PT-EN",
            font=ctk.CTkFont(size=28, weight="bold")
        )
        title_label.pack(pady=15)
        
        # Subtítulo
        subtitle_label = ctk.CTkLabel(
            header_frame,
            text="Tradução Automática em Tempo Real com IA",
            font=ctk.CTkFont(size=14),
            text_color="gray70"
        )
        subtitle_label.pack(pady=(0, 10))
    
    def create_video_section(self):
        """
        Cria seção de vídeo e controles
        """
        # Frame superior (vídeo + controles)
        top_frame = ctk.CTkFrame(self.main_frame)
        top_frame.pack(fill="x", pady=(0, 10))
        
        # Frame do vídeo (esquerda)
        video_frame = ctk.CTkFrame(top_frame)
        video_frame.pack(side="left", fill="both", expand=True, padx=(10, 5), pady=10)
        
        # Label do vídeo
        video_title = ctk.CTkLabel(
            video_frame,
            text="📹 Captura de Vídeo",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        video_title.pack(pady=(10, 5))
        
        # Canvas para vídeo
        self.video_canvas = tk.Canvas(
            video_frame,
            width=400,
            height=300,
            bg="#2b2b2b",
            highlightthickness=0
        )
        self.video_canvas.pack(pady=10)
        
        # Texto placeholder no canvas
        self.video_canvas.create_text(
            200, 150,
            text="📷 Câmera Desligada\n\nClique em 'Iniciar' para ativar",
            fill="white",
            font=("Arial", 12),
            justify="center"
        )
        
        # Frame de controles (direita)
        controls_frame = ctk.CTkFrame(top_frame)
        controls_frame.pack(side="right", fill="y", padx=(5, 10), pady=10)
        
        # Título dos controles
        controls_title = ctk.CTkLabel(
            controls_frame,
            text="🎛️ Controles",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        controls_title.pack(pady=(10, 15))
        
        # Botão principal (Iniciar/Parar)
        self.main_button = ctk.CTkButton(
            controls_frame,
            text="🚀 Iniciar Sistema",
            font=ctk.CTkFont(size=14, weight="bold"),
            width=200,
            height=50,
            command=self.toggle_system
        )
        self.main_button.pack(pady=10)
        
        # Seletor de direção de tradução
        direction_label = ctk.CTkLabel(
            controls_frame,
            text="🔄 Direção da Tradução:",
            font=ctk.CTkFont(size=12)
        )
        direction_label.pack(pady=(20, 5))
        
        self.direction_var = ctk.StringVar(value="pt_to_en")
        self.direction_menu = ctk.CTkOptionMenu(
            controls_frame,
            values=["PT → EN", "EN → PT", "Auto"],
            variable=self.direction_var,
            command=self.on_direction_change
        )
        self.direction_menu.pack(pady=5)
        
        # Switches para funcionalidades
        switches_frame = ctk.CTkFrame(controls_frame)
        switches_frame.pack(fill="x", padx=10, pady=20)
        
        # Switch Áudio
        self.audio_switch = ctk.CTkSwitch(
            switches_frame,
            text="🎤 Áudio",
            command=self.toggle_audio
        )
        self.audio_switch.pack(pady=5, anchor="w")
        self.audio_switch.select()  # Ativado por padrão
        
        # Switch Vídeo
        self.video_switch = ctk.CTkSwitch(
            switches_frame,
            text="📹 Vídeo",
            command=self.toggle_video
        )
        self.video_switch.pack(pady=5, anchor="w")
        self.video_switch.select()  # Ativado por padrão
        
        # Switch TTS
        self.tts_switch = ctk.CTkSwitch(
            switches_frame,
            text="🔊 Síntese de Voz",
            command=self.toggle_tts
        )
        self.tts_switch.pack(pady=5, anchor="w")
        self.tts_switch.select()  # Ativado por padrão
        
        # Switch Legendas
        self.subtitles_switch = ctk.CTkSwitch(
            switches_frame,
            text="📺 Legendas",
            command=self.toggle_subtitles
        )
        self.subtitles_switch.pack(pady=5, anchor="w")
        self.subtitles_switch.select()  # Ativado por padrão
    
    def create_translation_section(self):
        """
        Cria seção de tradução
        """
        # Frame de tradução
        translation_frame = ctk.CTkFrame(self.main_frame)
        translation_frame.pack(fill="both", expand=True, pady=(0, 10))
        
        # Título da seção
        translation_title = ctk.CTkLabel(
            translation_frame,
            text="🌐 Tradução em Tempo Real",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        translation_title.pack(pady=(10, 5))
        
        # Frame para textos (lado a lado)
        texts_frame = ctk.CTkFrame(translation_frame)
        texts_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Texto original (esquerda)
        original_frame = ctk.CTkFrame(texts_frame)
        original_frame.pack(side="left", fill="both", expand=True, padx=(0, 5))
        
        original_label = ctk.CTkLabel(
            original_frame,
            text="🎤 Texto Original",
            font=ctk.CTkFont(size=14, weight="bold")
        )
        original_label.pack(pady=(10, 5))
        
        self.original_text = ctk.CTkTextbox(
            original_frame,
            height=150,
            font=ctk.CTkFont(size=12),
            wrap="word"
        )
        self.original_text.pack(fill="both", expand=True, padx=10, pady=(0, 10))
        self.original_text.insert("1.0", "Aguardando entrada de áudio...\n\nFale algo para começar a tradução.")
        
        # Tradução (direita)
        translation_text_frame = ctk.CTkFrame(texts_frame)
        translation_text_frame.pack(side="right", fill="both", expand=True, padx=(5, 0))
        
        translation_label = ctk.CTkLabel(
            translation_text_frame,
            text="🌍 Tradução",
            font=ctk.CTkFont(size=14, weight="bold")
        )
        translation_label.pack(pady=(10, 5))
        
        self.translation_text = ctk.CTkTextbox(
            translation_text_frame,
            height=150,
            font=ctk.CTkFont(size=12),
            wrap="word"
        )
        self.translation_text.pack(fill="both", expand=True, padx=10, pady=(0, 10))
        self.translation_text.insert("1.0", "A tradução aparecerá aqui...\n\nWaiting for audio input to start translation.")
    
    def create_controls_section(self):
        """
        Cria seção de controles adicionais
        """
        controls_frame = ctk.CTkFrame(self.main_frame)
        controls_frame.pack(fill="x", pady=(0, 10))
        
        # Botões de ação
        buttons_frame = ctk.CTkFrame(controls_frame)
        buttons_frame.pack(fill="x", padx=10, pady=10)
        
        # Botão limpar
        clear_button = ctk.CTkButton(
            buttons_frame,
            text="🗑️ Limpar Textos",
            width=120,
            command=self.clear_texts
        )
        clear_button.pack(side="left", padx=5)
        
        # Botão salvar
        save_button = ctk.CTkButton(
            buttons_frame,
            text="💾 Salvar Histórico",
            width=120,
            command=self.save_history
        )
        save_button.pack(side="left", padx=5)
        
        # Botão configurações
        settings_button = ctk.CTkButton(
            buttons_frame,
            text="⚙️ Configurações",
            width=120,
            command=self.open_settings
        )
        settings_button.pack(side="left", padx=5)
        
        # Botão sobre
        about_button = ctk.CTkButton(
            buttons_frame,
            text="ℹ️ Sobre",
            width=120,
            command=self.show_about
        )
        about_button.pack(side="right", padx=5)
    
    def create_status_section(self):
        """
        Cria barra de status
        """
        status_frame = ctk.CTkFrame(self.main_frame)
        status_frame.pack(fill="x")
        
        # Status principal
        self.status_label = ctk.CTkLabel(
            status_frame,
            text="🔴 Sistema Parado | Pronto para iniciar",
            font=ctk.CTkFont(size=12)
        )
        self.status_label.pack(side="left", padx=10, pady=5)
        
        # Estatísticas
        self.stats_label = ctk.CTkLabel(
            status_frame,
            text="📊 Traduções: 0 | Tempo: 00:00:00",
            font=ctk.CTkFont(size=12),
            text_color="gray70"
        )
        self.stats_label.pack(side="right", padx=10, pady=5)

print("🎨 Interface principal criada!")

## 2. Métodos de Controle da Interface

In [ ]:
class NeuroTranslatorGUI(NeuroTranslatorGUI):
    """
    Extensão da classe com métodos de controle
    """
    
    def toggle_system(self):
        """
        Liga/desliga o sistema principal
        """
        if not self.is_running:
            self.start_system()
        else:
            self.stop_system()
    
    def start_system(self):
        """
        Inicia o sistema NeuroTranslator
        """
        try:
            print("🚀 Iniciando sistema NeuroTranslator...")
            
            # Atualizar interface
            self.main_button.configure(text="⏹️ Parar Sistema")
            self.status_label.configure(text="🟢 Sistema Ativo | Processando...")
            
            # Inicializar sistema (simulado)
            self.is_running = True
            self.start_time = datetime.now()
            
            # Iniciar threads de processamento
            self.start_video_thread()
            self.start_processing_thread()
            
            # Atualizar textos
            self.original_text.delete("1.0", "end")
            self.original_text.insert("1.0", "🎤 Sistema ativo! Fale algo...")
            
            self.translation_text.delete("1.0", "end")
            self.translation_text.insert("1.0", "🌐 Aguardando tradução...")
            
            print("✅ Sistema iniciado com sucesso!")
            
        except Exception as e:
            print(f"❌ Erro ao iniciar sistema: {e}")
            messagebox.showerror("Erro", f"Falha ao iniciar sistema:\n{e}")
    
    def stop_system(self):
        """
        Para o sistema NeuroTranslator
        """
        try:
            print("⏹️ Parando sistema NeuroTranslator...")
            
            # Parar sistema
            self.is_running = False
            
            # Atualizar interface
            self.main_button.configure(text="🚀 Iniciar Sistema")
            self.status_label.configure(text="🔴 Sistema Parado | Pronto para iniciar")
            
            # Limpar canvas de vídeo
            self.video_canvas.delete("all")
            self.video_canvas.create_text(
                200, 150,
                text="📷 Câmera Desligada\n\nClique em 'Iniciar' para ativar",
                fill="white",
                font=("Arial", 12),
                justify="center"
            )
            
            print("✅ Sistema parado com sucesso!")
            
        except Exception as e:
            print(f"❌ Erro ao parar sistema: {e}")
    
    def start_video_thread(self):
        """
        Inicia thread de captura de vídeo
        """
        if self.video_switch.get():
            self.video_thread = threading.Thread(target=self.video_loop)
            self.video_thread.daemon = True
            self.video_thread.start()
    
    def video_loop(self):
        """
        Loop de captura e exibição de vídeo
        """
        try:
            # Simular captura de vídeo
            cap = cv2.VideoCapture(0)
            
            if not cap.isOpened():
                print("⚠️ Câmera não encontrada, usando vídeo simulado")
                self.simulate_video()
                return
            
            while self.is_running and self.video_switch.get():
                ret, frame = cap.read()
                
                if ret:
                    # Redimensionar frame
                    frame = cv2.resize(frame, (400, 300))
                    
                    # Converter BGR para RGB
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    
                    # Converter para PIL Image
                    pil_image = Image.fromarray(frame_rgb)
                    
                    # Converter para PhotoImage
                    photo = ImageTk.PhotoImage(pil_image)
                    
                    # Atualizar canvas na thread principal
                    self.root.after(0, self.update_video_canvas, photo)
                
                time.sleep(0.033)  # ~30 FPS
            
            cap.release()
            
        except Exception as e:
            print(f"❌ Erro no vídeo: {e}")
            self.simulate_video()
    
    def simulate_video(self):
        """
        Simula vídeo quando câmera não está disponível
        """
        frame_count = 0
        
        while self.is_running and self.video_switch.get():
            # Criar frame simulado
            frame = np.zeros((300, 400, 3), dtype=np.uint8)
            frame[:] = (40, 40, 40)  # Cor de fundo escura
            
            # Adicionar texto
            cv2.putText(frame, "Camera Simulation", (100, 140), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(frame, f"Frame: {frame_count}", (150, 170), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
            
            # Adicionar círculo animado
            center_x = 200 + int(50 * np.sin(frame_count * 0.1))
            center_y = 200 + int(30 * np.cos(frame_count * 0.1))
            cv2.circle(frame, (center_x, center_y), 20, (0, 255, 0), -1)
            
            # Converter para PIL e PhotoImage
            pil_image = Image.fromarray(frame)
            photo = ImageTk.PhotoImage(pil_image)
            
            # Atualizar canvas
            self.root.after(0, self.update_video_canvas, photo)
            
            frame_count += 1
            time.sleep(0.1)
    
    def update_video_canvas(self, photo):
        """
        Atualiza canvas de vídeo (thread-safe)
        """
        try:
            self.video_canvas.delete("all")
            self.video_canvas.create_image(200, 150, image=photo)
            self.current_frame = photo  # Manter referência
        except Exception as e:
            print(f"❌ Erro ao atualizar vídeo: {e}")
    
    def start_processing_thread(self):
        """
        Inicia thread de processamento de tradução
        """
        self.processing_thread = threading.Thread(target=self.processing_loop)
        self.processing_thread.daemon = True
        self.processing_thread.start()
    
    def processing_loop(self):
        """
        Loop principal de processamento
        """
        translation_count = 0
        
        # Frases de exemplo para simulação
        example_phrases = [
            ("Olá, como você está?", "Hello, how are you?"),
            ("Bom dia! Tudo bem?", "Good morning! Everything okay?"),
            ("Obrigado pela ajuda", "Thank you for the help"),
            ("Até logo, tchau!", "See you later, bye!"),
            ("Como foi seu dia?", "How was your day?"),
            ("Hello, nice to meet you", "Olá, prazer em conhecê-lo"),
            ("Good afternoon everyone", "Boa tarde pessoal"),
            ("Thank you very much", "Muito obrigado"),
            ("Have a great day!", "Tenha um ótimo dia!"),
            ("See you tomorrow", "Até amanhã")
        ]
        
        while self.is_running:
            try:
                if self.audio_switch.get():
                    # Simular tradução a cada 5 segundos
                    time.sleep(5)
                    
                    if self.is_running:
                        # Escolher frase aleatória
                        import random
                        original, translation = random.choice(example_phrases)
                        
                        # Atualizar interface
                        self.root.after(0, self.update_translation, original, translation)
                        
                        translation_count += 1
                        
                        # Atualizar estatísticas
                        self.root.after(0, self.update_stats, translation_count)
                else:
                    time.sleep(1)
                    
            except Exception as e:
                print(f"❌ Erro no processamento: {e}")
                time.sleep(1)
    
    def update_translation(self, original, translation):
        """
        Atualiza textos de tradução (thread-safe)
        """
        try:
            timestamp = datetime.now().strftime("%H:%M:%S")
            
            # Atualizar texto original
            self.original_text.delete("1.0", "end")
            self.original_text.insert("1.0", f"[{timestamp}] {original}")
            
            # Atualizar tradução
            self.translation_text.delete("1.0", "end")
            self.translation_text.insert("1.0", f"[{timestamp}] {translation}")
            
            # Adicionar ao histórico
            self.translation_history.append({
                'timestamp': timestamp,
                'original': original,
                'translation': translation
            })
            
            print(f"🔄 [{timestamp}] {original} → {translation}")
            
        except Exception as e:
            print(f"❌ Erro ao atualizar tradução: {e}")
    
    def update_stats(self, translation_count):
        """
        Atualiza estatísticas (thread-safe)
        """
        try:
            if hasattr(self, 'start_time'):
                runtime = datetime.now() - self.start_time
                runtime_str = str(runtime).split('.')[0]  # Remove microsegundos
            else:
                runtime_str = "00:00:00"
            
            self.stats_label.configure(
                text=f"📊 Traduções: {translation_count} | Tempo: {runtime_str}"
            )
            
        except Exception as e:
            print(f"❌ Erro ao atualizar stats: {e}")

print("🎛️ Métodos de controle implementados!")

## 3. Métodos de Interface e Eventos

In [ ]:
class NeuroTranslatorGUI(NeuroTranslatorGUI):
    """
    Extensão com métodos de interface e eventos
    """
    
    def on_direction_change(self, value):
        """
        Callback para mudança de direção de tradução
        """
        direction_map = {
            "PT → EN": "pt_to_en",
            "EN → PT": "en_to_pt",
            "Auto": "auto"
        }
        
        direction = direction_map.get(value, "auto")
        print(f"🔄 Direção alterada para: {direction}")
        
        # Atualizar status
        if self.is_running:
            self.status_label.configure(
                text=f"🟢 Sistema Ativo | Direção: {value}"
            )
    
    def toggle_audio(self):
        """
        Liga/desliga processamento de áudio
        """
        enabled = self.audio_switch.get()
        print(f"🎤 Áudio {'habilitado' if enabled else 'desabilitado'}")
        
        if not enabled and self.is_running:
            self.original_text.delete("1.0", "end")
            self.original_text.insert("1.0", "🔇 Áudio desabilitado")
    
    def toggle_video(self):
        """
        Liga/desliga captura de vídeo
        """
        enabled = self.video_switch.get()
        print(f"📹 Vídeo {'habilitado' if enabled else 'desabilitado'}")
        
        if not enabled:
            self.video_canvas.delete("all")
            self.video_canvas.create_text(
                200, 150,
                text="📵 Vídeo Desabilitado",
                fill="gray",
                font=("Arial", 14),
                justify="center"
            )
    
    def toggle_tts(self):
        """
        Liga/desliga síntese de voz
        """
        enabled = self.tts_switch.get()
        print(f"🔊 TTS {'habilitado' if enabled else 'desabilitado'}")
    
    def toggle_subtitles(self):
        """
        Liga/desliga sistema de legendas
        """
        enabled = self.subtitles_switch.get()
        print(f"📺 Legendas {'habilitadas' if enabled else 'desabilitadas'}")
        
        if enabled and self.is_running:
            # Simular abertura de janela de legendas
            print("📺 Abrindo janela de legendas...")
    
    def clear_texts(self):
        """
        Limpa textos de tradução
        """
        self.original_text.delete("1.0", "end")
        self.translation_text.delete("1.0", "end")
        
        if self.is_running:
            self.original_text.insert("1.0", "🎤 Sistema ativo! Fale algo...")
            self.translation_text.insert("1.0", "🌐 Aguardando tradução...")
        else:
            self.original_text.insert("1.0", "Aguardando entrada de áudio...")
            self.translation_text.insert("1.0", "A tradução aparecerá aqui...")
        
        print("🗑️ Textos limpos")
    
    def save_history(self):
        """
        Salva histórico de traduções
        """
        if not self.translation_history:
            messagebox.showinfo("Info", "Nenhuma tradução para salvar.")
            return
        
        try:
            # Abrir diálogo para salvar arquivo
            filename = filedialog.asksaveasfilename(
                defaultextension=".json",
                filetypes=[("JSON files", "*.json"), ("All files", "*.*")],
                title="Salvar Histórico de Traduções"
            )
            
            if filename:
                with open(filename, 'w', encoding='utf-8') as f:
                    json.dump(self.translation_history, f, indent=2, ensure_ascii=False)
                
                messagebox.showinfo("Sucesso", f"Histórico salvo em:\n{filename}")
                print(f"💾 Histórico salvo: {filename}")
        
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar arquivo:\n{e}")
            print(f"❌ Erro ao salvar: {e}")
    
    def open_settings(self):
        """
        Abre janela de configurações
        """
        settings_window = SettingsWindow(self.root)
        print("⚙️ Janela de configurações aberta")
    
    def show_about(self):
        """
        Mostra informações sobre o aplicativo
        """
        about_text = """
🧠 NeuroTranslator PT-EN
Versão 1.0.0

Sistema de tradução automática em tempo real
usando Inteligência Artificial.

Funcionalidades:
• Reconhecimento de voz em tempo real
• Tradução automática PT ↔ EN
• Síntese de voz das traduções
• Captura de vídeo com detecção facial
• Sistema de legendas overlay

Desenvolvido com:
• Python + CustomTkinter
• OpenCV + TensorFlow
• Transformers + PyTorch

© 2025 - Projeto Experimental
        """
        
        messagebox.showinfo("Sobre o NeuroTranslator", about_text)
    
    def on_closing(self):
        """
        Callback para fechamento da aplicação
        """
        if self.is_running:
            if messagebox.askokcancel("Sair", "O sistema está ativo. Deseja realmente sair?"):
                self.stop_system()
                self.root.destroy()
        else:
            self.root.destroy()
    
    def run(self):
        """
        Inicia a aplicação
        """
        print("🚀 Iniciando interface NeuroTranslator...")
        self.root.mainloop()
        print("👋 Interface fechada")

print("🎮 Métodos de interface implementados!")

## 4. Janela de Configurações

In [ ]:
class SettingsWindow:
    """
    Janela de configurações do NeuroTranslator
    """
    def __init__(self, parent):
        self.parent = parent
        
        # Criar janela
        self.window = ctk.CTkToplevel(parent)
        self.window.title("⚙️ Configurações - NeuroTranslator")
        self.window.geometry("500x600")
        
        # Centralizar janela
        self.center_window()
        
        # Tornar modal
        self.window.transient(parent)
        self.window.grab_set()
        
        # Criar widgets
        self.create_widgets()
    
    def center_window(self):
        """
        Centraliza janela na tela
        """
        self.window.update_idletasks()
        width = self.window.winfo_width()
        height = self.window.winfo_height()
        x = (self.window.winfo_screenwidth() // 2) - (width // 2)
        y = (self.window.winfo_screenheight() // 2) - (height // 2)
        self.window.geometry(f"{width}x{height}+{x}+{y}")
    
    def create_widgets(self):
        """
        Cria widgets da janela de configurações
        """
        # Frame principal
        main_frame = ctk.CTkFrame(self.window)
        main_frame.pack(fill="both", expand=True, padx=20, pady=20)
        
        # Título
        title_label = ctk.CTkLabel(
            main_frame,
            text="⚙️ Configurações do Sistema",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        title_label.pack(pady=(10, 20))
        
        # Seção Áudio
        self.create_audio_section(main_frame)
        
        # Seção Vídeo
        self.create_video_section(main_frame)
        
        # Seção TTS
        self.create_tts_section(main_frame)
        
        # Seção Tradução
        self.create_translation_section(main_frame)
        
        # Botões
        self.create_buttons(main_frame)
    
    def create_audio_section(self, parent):
        """
        Cria seção de configurações de áudio
        """
        audio_frame = ctk.CTkFrame(parent)
        audio_frame.pack(fill="x", pady=10)
        
        # Título da seção
        audio_title = ctk.CTkLabel(
            audio_frame,
            text="🎤 Configurações de Áudio",
            font=ctk.CTkFont(size=14, weight="bold")
        )
        audio_title.pack(pady=(10, 5))
        
        # Taxa de amostragem
        sample_rate_label = ctk.CTkLabel(audio_frame, text="Taxa de Amostragem:")
        sample_rate_label.pack(pady=2)
        
        self.sample_rate_var = ctk.StringVar(value="16000")
        sample_rate_menu = ctk.CTkOptionMenu(
            audio_frame,
            values=["8000", "16000", "22050", "44100"],
            variable=self.sample_rate_var
        )
        sample_rate_menu.pack(pady=5)
        
        # Limiar de silêncio
        silence_label = ctk.CTkLabel(audio_frame, text="Sensibilidade do Microfone:")
        silence_label.pack(pady=(10, 2))
        
        self.silence_slider = ctk.CTkSlider(
            audio_frame,
            from_=100,
            to=1000,
            number_of_steps=18
        )
        self.silence_slider.set(500)
        self.silence_slider.pack(pady=5)
    
    def create_video_section(self, parent):
        """
        Cria seção de configurações de vídeo
        """
        video_frame = ctk.CTkFrame(parent)
        video_frame.pack(fill="x", pady=10)
        
        # Título da seção
        video_title = ctk.CTkLabel(
            video_frame,
            text="📹 Configurações de Vídeo",
            font=ctk.CTkFont(size=14, weight="bold")
        )
        video_title.pack(pady=(10, 5))
        
        # Resolução
        resolution_label = ctk.CTkLabel(video_frame, text="Resolução:")
        resolution_label.pack(pady=2)
        
        self.resolution_var = ctk.StringVar(value="640x480")
        resolution_menu = ctk.CTkOptionMenu(
            video_frame,
            values=["320x240", "640x480", "800x600", "1024x768"],
            variable=self.resolution_var
        )
        resolution_menu.pack(pady=5)
        
        # FPS
        fps_label = ctk.CTkLabel(video_frame, text="Taxa de Quadros (FPS):")
        fps_label.pack(pady=(10, 2))
        
        self.fps_var = ctk.StringVar(value="30")
        fps_menu = ctk.CTkOptionMenu(
            video_frame,
            values=["15", "24", "30", "60"],
            variable=self.fps_var
        )
        fps_menu.pack(pady=5)
    
    def create_tts_section(self, parent):
        """
        Cria seção de configurações de TTS
        """
        tts_frame = ctk.CTkFrame(parent)
        tts_frame.pack(fill="x", pady=10)
        
        # Título da seção
        tts_title = ctk.CTkLabel(
            tts_frame,
            text="🔊 Configurações de Síntese de Voz",
            font=ctk.CTkFont(size=14, weight="bold")
        )
        tts_title.pack(pady=(10, 5))
        
        # Volume
        volume_label = ctk.CTkLabel(tts_frame, text="Volume:")
        volume_label.pack(pady=2)
        
        self.volume_slider = ctk.CTkSlider(
            tts_frame,
            from_=0.0,
            to=1.0,
            number_of_steps=10
        )
        self.volume_slider.set(0.8)
        self.volume_slider.pack(pady=5)
        
        # Velocidade
        speed_label = ctk.CTkLabel(tts_frame, text="Velocidade da Fala:")
        speed_label.pack(pady=(10, 2))
        
        self.speed_slider = ctk.CTkSlider(\n            tts_frame,\n            from_=50,\n            to=300,\n            number_of_steps=25,\n            command=self.on_speed_change\n        )\n        self.speed_slider.set(150)\n        self.speed_slider.pack(pady=5)\n        \n        print(\"✅ Interface de configuração de TTS criada com sucesso!\")